In [1]:
from pyrocko import util, model, io, trace, moment_tensor, gmtpy
from pyrocko import pz
from pyrocko import orthodrome as od
from pyrocko.io import quakeml
from pyrocko.io import stationxml as fdsn
from pyrocko.client import catalog
from pyrocko.automap import Map
import pyrocko.moment_tensor as pmt
from seiscloud import plot as scp
from seiscloud import cluster as scc
import numpy as num
import os, sys, re, math, shutil
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
from matplotlib import dates
import datetime
import urllib.request
from pyrocko.plot.gmtpy import GMT

In [2]:
workdir='../'
catdir =  os.path.join(workdir,'CAT')

## GOSSIP: Events where either lat, long, depth or mag are not present

In [ ]:
catname = os.path.join(catdir, 'catologue_flegrei_GOSSIP_complete.pf')

cat_c = model.load_events(catname)
print('Number of events:', len(cat_c))

In [ ]:
magnitude_missed=[]
time=[]
for ev in cat_c:
    if ev.magnitude==None or ev.lat==None or ev.lon==None or ev.depth==None:
        magnitude_missed.append(ev.magnitude)
        time.append(ev.time)

eq_dates = [datetime.datetime.fromtimestamp(t) for t in time]

fig, axs = plt.subplots(1, 1, figsize=(17,6), sharex=True)
axs.scatter(eq_dates,magnitude_missed, s=5, color='green')
axs.set_ylabel('Magnitude')
axs.grid(True)
axs.set_xlabel('Time')


max=0
for mag in magnitude_missed:
    if mag != None:
        if mag>max:
            max=mag
print('max magnitude lost:',max)



## new catalogue with events over a certain magnitude

### GOSSIP

In [ ]:
catname = os.path.join(catdir, 'catologue_flegrei_GOSSIP.pf')

cat = model.load_events(catname)
print('Number of events:', len(cat))

In [ ]:
events_mag = []

minimum_magnitude=2.5

for ev in cat:
    if ev.magnitude>=minimum_magnitude :
        events_mag.append(model.Event(name=ev.name, time=ev.time,
                                lat=ev.lat, lon=ev.lon,
                                depth=ev.depth, magnitude=ev.magnitude))
events_mag.sort(key=lambda x: x.time, reverse=False)

catname = os.path.join(catdir, 'catologue_flegrei_GOSSIP_mag2_5.pf')

model.dump_events(events_mag, catname)
print('Number of events:', len(events_mag))

### INGV

In [ ]:
catname = os.path.join(catdir, 'catologue_flegrei_INGV.pf')

cat = model.load_events(catname)
print('Number of events:', len(cat))

In [ ]:
events_mag = []

minimum_magnitude=2.5

for ev in cat:
    if ev.magnitude>=minimum_magnitude :
        events_mag.append(model.Event(name=ev.name, time=ev.time,
                                lat=ev.lat, lon=ev.lon,
                                depth=ev.depth, magnitude=ev.magnitude))
events_mag.sort(key=lambda x: x.time, reverse=False)

catname = os.path.join(catdir, 'catologue_flegrei_INGV_mag2_5.pf')

model.dump_events(events_mag, catname)
print('Number of events:', len(events_mag))

# compare the two catalogues

In [3]:
catname = os.path.join(catdir, 'catologue_flegrei_GOSSIP_mag2_5.pf')

catalogue_G = model.load_events(catname)
print('Number of events GOSSIP:', len(catalogue_G))

catname = os.path.join(catdir, 'catologue_flegrei_GOSSIP.pf')

catalogue_G_complete = model.load_events(catname)
print('Number of events GOSSIP complete:', len(catalogue_G_complete))

catname = os.path.join(catdir, 'catologue_flegrei_INGV_mag2_5.pf')

catalogue_I = model.load_events(catname)
print('Number of events INGV:', len(catalogue_I))

catname = os.path.join(catdir, 'catologue_flegrei_INGV.pf')

catalogue_I_complete = model.load_events(catname)
print('Number of events INGV complete:', len(catalogue_I_complete))

Number of events GOSSIP: 51
Number of events GOSSIP complete: 8031
Number of events INGV: 43
Number of events INGV complete: 758


In [ ]:
id_catalogues=[]

for ind,ev_G in enumerate(catalogue_G):
    
    origin_time = ev_G.time
    time_min= origin_time - 5
    time_max= origin_time + 5

    id_catalogues.append([])
    id_catalogues[ind].append( ev_G.name )

    for ev_I in catalogue_I:
        if ev_I.time>time_min and ev_I.time<time_max:
            id_catalogues[ind].append( ev_I.name )
    

id_catalogues

In [ ]:
id_catalogues2=[]

for ind,ev_I in enumerate(catalogue_I):
    
    origin_time = ev_I.time
    time_min= origin_time - 5
    time_max= origin_time + 5

    id_catalogues2.append([])
    id_catalogues2[ind].append( ev_I.name )

    for ev_G in catalogue_G:
        if ev_G.time>time_min and ev_G.time<time_max:
            id_catalogues2[ind].append( ev_G.name )
    

id_catalogues2

In [ ]:
########################################################################
################OLD CATALOGUE########################################
########################################################################

events = []

#create a new catalogue with all the events from GOSSIP
for ev_G in catalogue_G:
#    events.append(ev_G)
#    ev_G.tags = ['GOSSIP_id:'+ str(ev_G.name) + ' mag:' + str(ev_G.magnitude)]
#    strtime = util.time_to_str(ev_G.time)
#    ev_G.name = 'flegrei_' + strtime[0:4] +'_'+ strtime[5:7]+'_'+strtime[8:10]+'_'+strtime[11:13]+'_'+strtime[14:16]+'_'+strtime[17:19]
    tag = ['GOSSIP_id:'+ str(ev_G.name) + ', mag:' + str(ev_G.magnitude)]
    strtime = util.time_to_str(ev_G.time)
    event_name = 'flegrei_' + strtime[0:4] +'_'+ strtime[5:7]+'_'+strtime[8:10]+'_'+strtime[11:13]+'_'+strtime[14:16]+'_'+strtime[17:19]
    events.append(model.Event(name=event_name, time=ev_G.time,
                            lat=ev_G.lat, lon=ev_G.lon,
                            depth=ev_G.depth,tags=tag))


events.sort(key=lambda x: x.time, reverse=False)
catname = os.path.join(catdir, 'catologue_flegrei_mag2_5.pf')
model.dump_events(events, catname)

events=model.load_events(catname)

#add the tags on the common events with INGV
ev_to_remove=[]

for ind,ev_G in enumerate(events):
    time_min= ev_G.time -5
    time_max= ev_G.time +5
    count=0
    for ev_I in catalogue_I:
        if ev_I.time>time_min and ev_I.time<time_max:
            ev_G.tags.append('INGV_id:'+ str(ev_I.name) + ', mag:' + str(ev_I.magnitude))
            count+=1
    if count!=1:
        ev_to_remove.append(ind)

for ind in reversed(ev_to_remove):
    del events[ind]

events.sort(key=lambda x: x.time, reverse=False)


catname = os.path.join(catdir, 'catologue_flegrei_mag2_5.pf')
model.dump_events(events, catname)
print('Number of events of merged catalogues:', len(events))

In [4]:
########################################################################
################ NEW CATALOGUE PART 1 ########################################
########################################################################

events = []

#create a new catalogue with all the events from GOSSIP
for ev_G in catalogue_G:
#    events.append(ev_G)
#    ev_G.tags = ['GOSSIP_id:'+ str(ev_G.name) + ' mag:' + str(ev_G.magnitude)]
#    strtime = util.time_to_str(ev_G.time)
#    ev_G.name = 'flegrei_' + strtime[0:4] +'_'+ strtime[5:7]+'_'+strtime[8:10]+'_'+strtime[11:13]+'_'+strtime[14:16]+'_'+strtime[17:19]
    tag = ['GOSSIP_id:'+ str(ev_G.name) + ', mag:' + str(ev_G.magnitude)]
    strtime = util.time_to_str(ev_G.time)
    event_name = 'flegrei_' + strtime[0:4] +'_'+ strtime[5:7]+'_'+strtime[8:10]+'_'+strtime[11:13]+'_'+strtime[14:16]+'_'+strtime[17:19]
    events.append(model.Event(name=event_name, time=ev_G.time,
                            lat=ev_G.lat, lon=ev_G.lon,
                            depth=ev_G.depth,tags=tag))


events.sort(key=lambda x: x.time, reverse=False)

#add the tags on the common events with INGV complete

ev_to_remove=[]

for ind,ev_G in enumerate(events):
    time_min= ev_G.time -5
    time_max= ev_G.time +5
    count=0
    for ev_I in catalogue_I_complete:
        if ev_I.time>time_min and ev_I.time<time_max:
            ev_G.tags.append('INGV_id:'+ str(ev_I.name) + ', mag:' + str(ev_I.magnitude))
            count+=1
    if count!=1:
        ev_to_remove.append(ind)

#remove multiple events
for ind in reversed(ev_to_remove):
    del events[ind]

catname = os.path.join(catdir, 'catologue_flegrei_new_mag2_5.pf')
model.dump_events(events, catname)
print('Number of events of merged catalogues:', len(events))

Number of events of merged catalogues: 49


In [5]:
catname = os.path.join(catdir, 'catologue_flegrei_new_mag2_5.pf')
events=model.load_events(catname)
print('Number of events of merged catalogues:', len(events))

Number of events of merged catalogues: 49


In [6]:
########################################################################
################ NEW CATALOGUE PART 2 ########################################
########################################################################

events_to_append=[]

#add to catalogue events from INGV of mag > 2.5 not present in GOSSIP > 2.5 catalogue
for ev_I in catalogue_I:
    time_min= ev_I.time -5
    time_max= ev_I.time +5
    count=0
    for ev in events:
        if ev.time>time_min and ev.time<time_max:
            count+=1
    if count<1:
        tag = ['INGV_id:'+ str(ev_I.name) + ', mag:' + str(ev_I.magnitude)]
        strtime = util.time_to_str(ev_I.time)
        event_name = 'flegrei_' + strtime[0:4] +'_'+ strtime[5:7]+'_'+strtime[8:10]+'_'+strtime[11:13]+'_'+strtime[14:16]+'_'+strtime[17:19]
        events_to_append.append(model.Event(name=event_name, time=ev_I.time,
                            lat=ev_I.lat, lon=ev_I.lon,
                            depth=ev_I.depth,tags=tag))


events_to_append.sort(key=lambda x: x.time, reverse=False)

#add the tags on the common events with INGV complete

ev_to_remove=[]

for ind,ev_I in enumerate(events_to_append):
    time_min= ev_I.time -5
    time_max= ev_I.time +5
    count=0
    for ev_G in catalogue_G_complete:
        if ev_G.time>time_min and ev_G.time<time_max:
            ev_I.tags.append('GOSSIP_id:'+ str(ev_G.name) + ', mag:' + str(ev_G.magnitude))
            count+=1
    if count!=1:
        ev_to_remove.append(ind)

#remove multiple events
for ind in reversed(ev_to_remove):
    del events_to_append[ind]

for ev in events_to_append:
    events.append(ev)

events.sort(key=lambda x: x.time, reverse=False)


catname = os.path.join(catdir, 'catologue_flegrei_new_mag2_5.pf')
model.dump_events(events, catname)
print('Number of events of merged catalogues:', len(events))


Number of events of merged catalogues: 53


In [7]:
[ev.tags for ev in events]

[['GOSSIP_id:4217', 'mag:2.5', 'INGV_id:6142151', 'mag:2.5'],
 ['GOSSIP_id:677', 'mag:2.5', 'INGV_id:20713961', 'mag:2.1'],
 ['GOSSIP_id:1159', 'mag:2.5', 'INGV_id:21843321', 'mag:2.0'],
 ['GOSSIP_id:2086', 'mag:2.5', 'INGV_id:23222571', 'mag:2.3'],
 ['GOSSIP_id:2508', 'mag:3.1', 'INGV_id:23542681', 'mag:2.4'],
 ['GOSSIP_id:2943', 'mag:2.7', 'INGV_id:24236231', 'mag:2.1'],
 ['GOSSIP_id:3020', 'mag:2.7', 'INGV_id:24317251', 'mag:1.9'],
 ['GOSSIP_id:3022', 'mag:3.3', 'INGV_id:24317311', 'mag:2.4'],
 ['GOSSIP_id:4417', 'mag:2.6', 'INGV_id:25853021', 'mag:2.3'],
 ['GOSSIP_id:7102', 'mag:3.47', 'INGV_id:30205901', 'mag:2.7'],
 ['GOSSIP_id:14725', 'mag:3.57', 'INGV_id:30350681', 'mag:3.0'],
 ['GOSSIP_id:14885', 'mag:2.94', 'INGV_id:30671901', 'mag:2.3'],
 ['GOSSIP_id:16351', 'mag:2.51', 'INGV_id:32166291', 'mag:2.7'],
 ['GOSSIP_id:17434', 'mag:2.67', 'INGV_id:33361881', 'mag:2.7'],
 ['GOSSIP_id:17732', 'mag:2.53', 'INGV_id:33524091', 'mag:2.6'],
 ['GOSSIP_id:22730', 'mag:2.67', 'INGV_id:3374

### Load

In [ ]:
catname = os.path.join(catdir, 'catologue_flegrei_new_mag2_5.pf')

cat = model.load_events(catname)
print('Number of events:', len(cat))